In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
import sklearn

In [5]:
def compltedf():
    %matplotlib inline
    ''' This script is used to visualize the top stocks and their performace based on historical and real time data'''
    import warnings
    warnings.filterwarnings("ignore")
    import pandas as pd
    import os
    import sys
    import time
    import requests
    from datetime import datetime, timedelta
    ##########################################################################################
    timestr = time.strftime("%Y%m%d")
    mydate=time.strftime("%d%m%Y")
    s = mydate
    date = datetime.strptime(s, "%d%m%Y")
    if datetime.now().strftime('%A')=='Saturday':
        modified_date = date + timedelta(days=-1)
    elif datetime.now().strftime('%A')=='Sunday': 
        modified_date = date + timedelta(days=-2)
    else:
        modified_date=datetime.now()
    correct_date=datetime.strftime(modified_date, "%d%m%Y")
    print (correct_date)
    def data_Downloader(url,filename):
        try:
            data=requests.get(url)
            if data.status_code==200:
                datacsv=data.text

                with open(filename,'w') as handler:
                    handler.write(datacsv)
                print('File written to '+ filename)
                return 'success'
            else:
                return 'failed'
        except:
            print('Internet Not Available,check please ')
            pass

    niftyfileUrl='https://www.nseindia.com/content/indices/ind_nifty100list.csv'
    NiftyFile='NiftyFile.csv'
    volatility='https://www.nseindia.com/archives/nsccl/volt/CMVOLT_'+ correct_date +'.CSV'
    niftyout=data_Downloader(niftyfileUrl,NiftyFile)
    volatilityFile='Volatility_'+correct_date
    volout=data_Downloader(volatility,volatilityFile)
    if niftyout=='success' and volout=='success':
        print( 'data gathering complete')
    elif volout=='failed':
        modified_date=date + timedelta(days=-1)
        correct_date=datetime.strftime(modified_date, "%d%m%Y")
        volatility='https://www.nseindia.com/archives/nsccl/volt/CMVOLT_'+ correct_date +'.CSV'
        volout=data_Downloader(volatility,volatilityFile)
        if volout=='failed':
            modified_date=date + timedelta(days=-2)
            correct_date=datetime.strftime(modified_date, "%d%m%Y")
            volatility='https://www.nseindia.com/archives/nsccl/volt/CMVOLT_'+ correct_date +'.CSV'
            volout=data_Downloader(volatility,volatilityFile)
    else:

        print('can not gather data for today , plotting on yesterdays data ')

    #####################################################################################################################################
    ###########data gathering complete###################################################################################################
    #####################################################################################################################################

    ##################Begin data cleansing and tranformation# #######################
    #############################################################################
    #reading both files
    if  os.path.isfile(volatilityFile):
        df1 = pd.read_csv(NiftyFile, header=0 )
        df2 = pd.read_csv(volatilityFile, header=0)
    #merging both files
        df = pd.merge(df1, df2, on='Symbol')
        df.rename(columns={'Previous Day Underlying Volatility (D)': 'previous_volatility', 'Current Day Underlying Daily Volatility (E) = Sqrt(0.94*D*D + 0.06*C*C)': 'current_volatility','Underlying Annualised Volatility (F) = E*Sqrt(365)':'annual_volatility','Underlying Previous Day Close Price (B)':'previous_close','Underlying Close Price (A)':'current_close','Company Name':'company_name'}, inplace=True)
        df3 = df.sort_values(by=['annual_volatility', 'previous_volatility','current_volatility'])
        df3['annual_volatility'] = pd.to_numeric(df3['annual_volatility'],errors='coerce')
        df3['current_volatility'] = pd.to_numeric(df3['current_volatility'],errors='coerce')
        df3['previous_volatility'] = pd.to_numeric(df3['previous_volatility'],errors='coerce')
        df3['current_close'] = pd.to_numeric(df3['current_close'],errors='coerce')
        df3['previous_close'] = pd.to_numeric(df3['previous_close'],errors='coerce')
        df3['current_volatility']=df3['current_volatility'].apply(lambda x: x*100)
        df3['previous_volatility']=df3['previous_volatility'].apply(lambda x: x*100)
        df=df3.loc[df3['current_volatility'] >= 2.0]
        df4 = df3.sort_values('current_volatility',ascending=[False])
        df4['Date']=s
        df5=df4[['company_name','Date','current_close','previous_close','previous_volatility','current_volatility']]
        df5['close_diff']=df5['current_close']-df5['previous_close']
        df5['voldiff']=df5['current_volatility']-df5['previous_volatility']
        completedf=df5.drop(['current_close','previous_close','previous_volatility','current_volatility'],axis=1)
        completedf=completedf.reset_index()
        completedf=completedf.drop(['index'],axis=1)
        return completedf

In [6]:
df=pd.read_csv('/home/shawak/Downloads/completeData.csv')

In [24]:
company_names=df.company_name.unique()
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
completedf=compltedf()

30082019
File written to NiftyFile.csv
File written to Volatility_30082019
data gathering complete


In [54]:

flag=0
length=len(df)
for companies in company_names:
    print('working on company '+companies)
    companydf=df.loc[df['company_name']==companies]
    dfx=companydf[['close_diff','voldiff']]
    dfy=companydf[['target']]

    #print (dfy.target.unique())
    if len(dfy.target.unique())>1:
        model=LogisticRegression()
        model.fit(dfx,dfy)
        companytodaydata=completedf.loc[completedf['company_name']==companies]
        data=companytodaydata[['close_diff','voldiff']]
        #print (data)
        #print(model.predict(data))
        #print(model.score(dfx,dfy))
    
        companytodaydata['target']=model.predict(data)
        checkvar=model.predict(data)
    else:
        companytodaydata['target']=dfy.target.unique()[0]
        checkvar=dfy.target.unique()
        
    index=length+flag
    dflist=companytodaydata.values.tolist()
    
        
    with open('/home/shawak/Downloads/completeData.csv','a') as handler:
        handler.write(str(index)+','+dflist[0][0]+','+dflist[0][1]+','+str(dflist[0][2])+','+str(dflist[0][3])+','+str(checkvar[0]))
        handler.write('\n')
    flag=flag+1
    if int(checkvar[0])==0:
        print('Please do not put money on ==>'+companies)
    else:
        print('hurrah, you can put your money on ')
    

working on company Vodafone Idea Ltd.
Please do not put money on ==>Vodafone Idea Ltd.
working on company Tata Motors Ltd.
hurrah, you can put your money on 
working on company Indiabulls Housing Finance Ltd.
Please do not put money on ==>Indiabulls Housing Finance Ltd.
working on company Tata Motors Ltd DVR
Please do not put money on ==>Tata Motors Ltd DVR
working on company DLF Ltd.
Please do not put money on ==>DLF Ltd.
working on company Yes Bank Ltd.
Please do not put money on ==>Yes Bank Ltd.
working on company Zee Entertainment Enterprises Ltd.
Please do not put money on ==>Zee Entertainment Enterprises Ltd.
working on company InterGlobe Aviation Ltd.
hurrah, you can put your money on 
working on company Hindustan Petroleum Corporation Ltd.
Please do not put money on ==>Hindustan Petroleum Corporation Ltd.
working on company Steel Authority of India Ltd.
Please do not put money on ==>Steel Authority of India Ltd.
working on company Motherson Sumi Systems Ltd.
Please do not put m

hurrah, you can put your money on 
working on company Hindustan Zinc Ltd.
Please do not put money on ==>Hindustan Zinc Ltd.
working on company Britannia Industries Ltd.
hurrah, you can put your money on 
working on company Housing Development Finance Corporation Ltd.
Please do not put money on ==>Housing Development Finance Corporation Ltd.
working on company Bosch Ltd.
hurrah, you can put your money on 
working on company HDFC Bank Ltd.
Please do not put money on ==>HDFC Bank Ltd.
working on company Hindustan Unilever Ltd.
Please do not put money on ==>Hindustan Unilever Ltd.
